## Problem 3
In this part we study how different particles affect each other when moving around in a network in continuous time. We consider the open network of Fig. 2 (see text), with transition rate matrix Lambda_open (see Lambda_open in the code below).

In [ ]:
##### Setup
import numpy as np
from numpy.random import choice
from copy import deepcopy
import matplotlib.pyplot as plt

# Define the mapping between 0,1,2,3,4 and o,a,b,c,d
mapping = {"o'":0, 'o':1, 'a':2, 'b':3, 'c':4, 'd':5, "d'":6}
reverse_mapping = {0:"o'", 1:'o', 2:'a', 3:'b', 4:'c', 5:'d', 6:"d'"}

# Create Lambda_open
Lambda_open = np.array([
    [0, 2/3, 1/3, 0, 0],
    [0, 0, 1/4, 1/4, 2/4],
    [0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0]])

# Add two auxiliary nodes to the open network, to make it "closed": o' and d'
# This is done by adding two rows and columns to Lambda_open: those relative to o' and d'
# In other words, convert Lambda_open to Lambda_closed
Lambda_closed = deepcopy(Lambda_open)
col_o_prime = np.array([0, 0, 0, 0, 0]).reshape(-1,1)
col_d_prime = np.array([0, 0, 0, 0, 1]).reshape(-1,1)
Lambda_closed = np.hstack((col_o_prime, Lambda_closed, col_d_prime))
row_o_prime = [1, 0, 0, 0, 0, 0, 0]    # self-loop: particles in o' stay in o' (a separate clock regulates the entrance of the particles in the open network)
row_d_prime = [0, 0, 0, 0, 0, 0, 1]    # self-loop: particles in d' stay in d'
Lambda_closed = np.vstack((row_o_prime, Lambda_closed, row_d_prime))

# Define Q, the transition matrix associated to the jump chain associated to the CTMC on the closed network
w = np.sum(Lambda_closed, axis=1)
w_star = np.max(w)
Q = Lambda_closed/w_star
Q = Q + np.diag(np.ones(len(w))-np.sum(Q,axis=1))
print("Q matrix:\n", Q)


**a) Proportional rate**

In [ ]:
def simulate_open_network_proportional_rate(input_rate=1, time_limit=60):
    '''
    This function simulates particles entering in the open network (= moving from o' to o)
    every time a (input_rate)-rate Poisson clock ticks; each of these particles then performs
    a random walk on the open network, until it exits it (= moves from d to d').
    Each node of the open network passes along particles at each tick of a Poisson clock
    with rate proportional to the n. of particles on it.
    
    The simulation stops after t=60 time units have passed.
    
    Returns:
        node_distribution: a list of 7-tuples;
                           node_distribution[i] is the distribution of the 100 particles on the len(Q)=7 nodes
                           of the closed network, in the time interval ( transition_times[i], transition_times[i+1] )
        transition_times: a list of values;
                          transition_times[i], for i!=-1, is the time instant in which the i-th transition happens
                          transition_times[-1] is t=60.
    '''
    
    # Compute at random the time instants in which a new particle enters in node 'o',
    # and therefore also the total number of particles which will enter during the simulation
    enter_times = []
    time = 0
    enter_t_next = -np.log(np.random.rand())/input_rate
    while time + enter_t_next <= time_limit:
        enter_times.append(time + enter_t_next)
        time = time + enter_t_next
        enter_t_next = -np.log(np.random.rand())/input_rate
    n_particles = len(enter_times)
    
    # Simulation
    n_nodes = len(Q) # =7
    time = 0    # global clock time
    transition_times = [0]    # this will store the time instants in which jumps are taken (i.e. in which the global clock ticks)
    rate = n_particles * w_star
    t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition is drawn from a rate-(100*w_star) exponential distribution

    node_distribution = [ [n_particles,0,0,0,0,0,0] ]    # a list which will contain lists, each containing seven values (the n. of particles in each of the seven nodes)

    while time + t_next <= time_limit:
        #current_node_distribution = deepcopy(node_distribution[-1])
        
        
        while len(enter_times) > 0 and time + t_next >= enter_times[0]:
            # The global clock has ticked after one (or more) particle have entered:
            # first let that (those) particle enter in node o, then eventually move a particle in the open network

            # A particle enters the open network in node o
            current_node_distribution = deepcopy(node_distribution[-1])
            current_node_distribution[0] -= 1
            current_node_distribution[1] += 1
            
            node_distribution.append(current_node_distribution)
            transition_times.append(enter_times[0])
            
            del enter_times[0]    # discard the enter time encountered
            # in whatever case, move a particle in the open network

        # Choose one of the 7 nodes at random (with probabilities proportional to the distribution of particles in the nodes);
        # the chosen node will pass along a particle
        current_node_distribution = deepcopy(node_distribution[-1])
        probs = [current_node_distribution[i] / n_particles for i in range(7)]
        node = choice(range(7), size=1, p=probs)[0]
        
        if current_node_distribution[node] > 0 and node!=0 and node!=6:
        # Random jump
        # if node node has at least a particle sojourning on it, and if it is not o' or d',
        # then move a particle from node node to a neighboring node, with probabilities
        # given by the row of Q corresponding to node node
            new_node = choice(range(7), size=1, p=Q[node])[0]
            new_node_distribution = deepcopy(current_node_distribution)
            new_node_distribution[node] -= 1
            new_node_distribution[new_node] += 1
            node_distribution.append(new_node_distribution)
            
            time = time + t_next    # time instant of the jump just occurred
            transition_times.append(time)

            t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition
        else:
            # otherwise, if node node has no particles sojourning on it, or if it is o' or d',
            # don't take any jumps and just update the time
            time = time + t_next
            t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition
            
            
    else:
        # no more transitions happen, but what about entrances?
        # there may be one or even more particles waiting to enter the open network
        while len(enter_times) > 0:
            time = enter_times.pop(0)
            transition_times.append(time)
            # A particle enters the open network in node o
            current_node_distribution = deepcopy(node_distribution[-1])
            current_node_distribution[0] -= 1
            current_node_distribution[1] += 1
            node_distribution.append(current_node_distribution)
            
        # Append the final distribution to node_distribution and 60 to transition_times
        # (this is done to correctly plot the n. of particles in each node during the simulation)
        node_distribution.append(node_distribution[-1])
        transition_times.append(time_limit)
        
    return (node_distribution, transition_times)


In [ ]:
# Perform the simulation with entrance rate = 1
node_distribution, transition_times = simulate_open_network_proportional_rate(input_rate=1)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()



# Perform the simulation with entrance rate = 10
node_distribution, transition_times = simulate_open_network_proportional_rate(input_rate=10)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()



# Perform the simulation with entrance rate = 100
node_distribution, transition_times = simulate_open_network_proportional_rate(input_rate=100)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(3, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()

**b) Fixed rate**

In [ ]:
def simulate_open_network_fixed_rate(input_rate=1, time_limit=60):
    '''
    This function simulates particles entering in the open network (= moving from o' to o)
    every time a (input_rate)-rate Poisson clock ticks; each of these particles then performs
    a random walk on the open network, until it exits it (= moves from d to d').
    Each node of the open network passes along particles, if any, at each tick of a Poisson clock
    with a fixed rate = 1, associated to that node.
    
    The simulation stops after t=60 time units have passed.
    
    Returns:
        node_distribution: a list of 7-tuples;
                           node_distribution[i] is the distribution of the 100 particles on the len(Q)=7 nodes
                           of the closed network, in the time interval ( transition_times[i], transition_times[i+1] )
        transition_times: a list of values;
                          transition_times[i], for i!=-1, is the time instant in which the i-th transition happens
                          transition_times[-1] is t=60.
    '''
    
    # Compute at random the time instants in which a new particle enters in node 'o',
    # and therefore also the total number of particles which will enter during the simulation
    enter_times = []
    time = 0
    enter_t_next = -np.log(np.random.rand())/input_rate
    while time + enter_t_next <= time_limit:
        enter_times.append(time + enter_t_next)
        time = time + enter_t_next
        enter_t_next = -np.log(np.random.rand())/input_rate
    n_particles = len(enter_times)
    
    # Simulation
    n_nodes = len(Q) # =7
    time = 0    # global clock time
    transition_times = [0]    # this will store the time instants in which jumps are taken (i.e. in which the global clock ticks)
    rate = 5 * w_star
    t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition is drawn from a rate-(100*w_star) exponential distribution

    node_distribution = [ [n_particles,0,0,0,0,0,0] ]    # a list which will contain lists, each containing seven values (the n. of particles in each of the seven nodes)

    while time + t_next <= time_limit:
        #current_node_distribution = deepcopy(node_distribution[-1])
        
        while len(enter_times) > 0 and time + t_next >= enter_times[0]:
            # The global clock has ticked after one (or more) particle have entered:
            # first let that (those) particle enter in node o, then eventually move a particle in the open network

            # A particle enters the open network in node o
            current_node_distribution = deepcopy(node_distribution[-1])
            current_node_distribution[0] -= 1
            current_node_distribution[1] += 1
            
            #print(current_node_distribution)
            node_distribution.append(current_node_distribution)
            transition_times.append(enter_times[0])
            
            del enter_times[0]    # discard the enter time encountered
            # in whatever case, move a particle in the open network

        # Choose one of the 5 nodes of the open network at random (uniformly);
        # the chosen node will pass along a particle, if it has one
        node = choice(range(1,6), size=1)[0]
        current_node_distribution = deepcopy(node_distribution[-1])
        
        if current_node_distribution[node] > 0:
        # Random jump
        # if node node has at least a particle sojourning on it, then move a particle from node node to a neighboring node,
        # with probabilities given by the row of Q corresponding to node node
            new_node = choice(range(7), size=1, p=Q[node])[0]
            new_node_distribution = deepcopy(node_distribution[-1])
            new_node_distribution[node] -= 1    # NB: it doesn't matter if node and new_node coincide
            new_node_distribution[new_node] += 1
            node_distribution.append(new_node_distribution)
            
            time = time + t_next    # time instant of the jump just occurred
            transition_times.append(time)

            t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition
        else:
            # otherwise, if node node has no particles sojourning on it, don't take any jumps and just update the time
            time = time + t_next
            t_next = -np.log(np.random.rand())/rate    # the random time to wait for the next transition
        
    else:
        # no more transitions happen, but what about entrances?
        # there may be one or even more particles waiting to enter the open network
        while len(enter_times) > 0:
            time = enter_times.pop(0)
            transition_times.append(time)
            # A particle enters the open network in node o
            current_node_distribution = deepcopy(node_distribution[-1])
            current_node_distribution[0] -= 1
            current_node_distribution[1] += 1
            node_distribution.append(current_node_distribution)
        
        # Append the final distribution to node_distribution and 60 to transition_times
        # (this is done to correctly plot the n. of particles in each node during the simulation)
        node_distribution.append(node_distribution[-1])
        transition_times.append(time_limit)
            
    return (node_distribution, transition_times)


In [ ]:
# Perform the simulation with entrance rate = 1
node_distribution, transition_times = simulate_open_network_fixed_rate(input_rate=1, time_limit=60)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(1, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()



# Perform the simulation with entrance rate = 2
node_distribution, transition_times = simulate_open_network_fixed_rate(input_rate=1.1, time_limit=60)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(2, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()



# Perform the simulation with entrance rate = 5
node_distribution, transition_times = simulate_open_network_fixed_rate(input_rate=1.2, time_limit=60)
node_distribution = np.array(node_distribution)

# Plot the evolution of the n. of particles in each node over time
fig = plt.figure(3, figsize=(16,8))
ax = plt.subplot()
for node in range(1,6):
    ax.plot(transition_times, node_distribution[:,node], label=f'node {reverse_mapping[node]}')    # label=f'node {reverse_mapping[node]}'
    
ax.legend()